In [1]:
# script to extract data from g adventures API


In [2]:
# do i need the full name for itinerary name?

In [3]:
# get promotional rates including promotion e.g. 25% of

In [4]:
from gapipy import Client
from datetime import timedelta
import pandas as pd

In [5]:
import re
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("G_ADVENTURES_KEY")

client = Client(application_key=api_key) 

In [6]:
filtered_departures = client.departures.filter(start_date="2024-02-13")

In [7]:
# filtered_departures = client.departures.filter(start_date__gt="2024-04-01", finish_date__lt="2024-04-30")

In [8]:
filtered_departures.count()

111

In [9]:
filtered_departures.first().to_dict()

{'id': '1131977',
 'href': 'https://rest.gadventures.com/departures/1131977',
 'date_created': '2021-01-12T09:48:44Z',
 'date_last_modified': '2023-10-03T10:13:57Z',
 'start_date': '2024-02-13',
 'finish_date': '2024-02-16',
 'date_cancelled': None,
 'flags': ['TWIN_SHARE', 'CHAT_ENABLED'],
 'availability': {'status': 'AVAILABLE', 'total': 15},
 'travel_ready_policy': {'code': 'NO_POLICY', 'name': 'No Policy'},
 'lowest_pp2a_prices': [{'currency': 'USD', 'amount': '879.20'},
  {'currency': 'AUD', 'amount': '1255.20'},
  {'currency': 'CHF', 'amount': '719.20'},
  {'currency': 'KRW', 'amount': '1163680.00'},
  {'currency': 'CNY', 'amount': '6367.20'},
  {'currency': 'JPY', 'amount': '127360.00'},
  {'currency': 'GBP', 'amount': '639.20'},
  {'currency': 'NZD', 'amount': '1359.20'},
  {'currency': 'EUR', 'amount': '743.20'},
  {'currency': 'ZAR', 'amount': '14879.20'},
  {'currency': 'SGD', 'amount': '1199.20'},
  {'currency': 'HKD', 'amount': '6895.20'},
  {'currency': 'CAD', 'amount': '

In [10]:
my_generator = filtered_departures.all() # .all()

In [11]:
departures_dictionaries_list = []
countries_set = set()
reference_count = 0
for value in my_generator:
    reference_count += 1
    if reference_count % 100 == 0:
        print(reference_count)
    try:
        if value.availability['status'] == "AVAILABLE" and value.tour_dossier != None: 
            trip_list = [category['name'] for category in value.tour_dossier.categories if category['category_type']['label'] ==  "Trip Type"]
            if 'Small Group' in trip_list:
                for i in range(len(value.structured_itineraries)):

                    # get tour name
                    tour_name = value.tour_dossier.name
                    print(tour_name)
                    print(value.id)

                    # structured itinerary id
                    structured_itineraries_name = value.structured_itineraries[i].name

                    # get the list of countries visited
                    visited_countries_list = []
                    for country in value.tour_dossier.geography['visited_countries']:
                        visited_countries_list.append(country["name"])
                        countries_set.add(country["name"]) #  add countries to set

                    
                    # get start date
                    start_date = value.start_date
                
                    # get duration
                    duration = value.finish_date - value.start_date + timedelta(days=1)
                    duration = duration.days

                    # get pricing form room categories
                    room_categories_dictionaries = {}
                    
                    for room in value.rooms:
                        data = room.to_dict()
                        for price_brand in data["price_bands"]:
                            for price in price_brand["prices"]:
                                if price['currency'] == "GBP":
                                    # get prices
                                    price_column = f'{data["name"]} - {price_brand["name"]}'
                                    if price_column in room_categories_dictionaries:
                                        room_categories_dictionaries[price_column] += price['amount']
                                    else:
                                        room_categories_dictionaries[price_column] = price['amount']
                                    # get promotion
                                    if  len(price['promotions']) > 0:
                                        print("started promo")
                                        # get promo cost
                                        promotion_column_cost = f'{data["name"]} - {price_brand["name"]} - Promotion cost'
                                        if promotion_column_cost in room_categories_dictionaries:
                                            room_categories_dictionaries[promotion_column_cost] += price["promotions"][0]['amount']
                                        else:
                                            room_categories_dictionaries[promotion_column_cost] = price["promotions"][0]['amount']
                                        # promo cost done
                                        # get promo description
                                        promotion_column_name = f'{data["name"]} - {price_brand["name"]} - Promotion description'
                                        if promotion_column_name in room_categories_dictionaries:
                                            room_categories_dictionaries[promotion_column_name] += client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                        else:
                                            room_categories_dictionaries[promotion_column_name] = client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                        print("finished promo")
                                            
                    # get tour url 
                    tour_url = value.tour_dossier.site_links[0]['href']
                    
                    # get itinerary day by day
                    itinerary = ""
                    itinerary_days = value.structured_itineraries[i]
                    for day in itinerary_days.days:
                        itinerary += f'Day {day.day}: {day.summary}\n'

                    # get tour categories
                    categories_id_list = [i['id'] for i in value.tour_dossier.categories]

                    tour_categories_dictionaries = {}

                    for category in categories_id_list:
                        category_dict = client.tour_categories.get(category).to_dict()
                        category_value = None
                        if category_dict['description'] == None:
                            category_value = category_dict['name']
                        else:
                            category_value = f"{category_dict['name']} - {category_dict['description']}"

                        if category_dict['category_type']['name'] in tour_categories_dictionaries:
                            tour_categories_dictionaries[category_dict['category_type']['name']] += ", " + category_value
                        else:
                            tour_categories_dictionaries[category_dict['category_type']['name']] = category_value

                    # create dictionary
                    dataframe_dictionary = {}
                    dataframe_dictionary['tour_name'] = tour_name
                    dataframe_dictionary['itinerary_name'] = structured_itineraries_name
                    dataframe_dictionary['visited_countries'] = ", ".join(visited_countries_list)
                    dataframe_dictionary['start_date'] = start_date
                    dataframe_dictionary['duration'] = duration
                    dataframe_dictionary['url'] = tour_url
                    dataframe_dictionary['itinerary'] = itinerary


                    # combine dictionaries
                    dataframe_dictionary = dict(dataframe_dictionary, **tour_categories_dictionaries, **room_categories_dictionaries)


                    
                    # add full dict to list
                    departures_dictionaries_list.append(dataframe_dictionary)
                    print(tour_name + " " + "done")
     
    except:
        print("403 issue")


Sailing Thailand - Ko Phi Phi to Phuket
1131977
started promo
finished promo
started promo
finished promo
Sailing Thailand - Ko Phi Phi to Phuket done
Masai Mara Camping Safari
1192691
Masai Mara Camping Safari done
Indochina Discovery: Seasides and Street Food
1201229
Indochina Discovery: Seasides and Street Food done
403 issue
403 issue
403 issue
Torres del Paine - The W Trek
1217529
Torres del Paine - The W Trek done
Hike Patagonia In Depth
1217820
Hike Patagonia In Depth done
Patagonia Hiking
1217865
Patagonia Hiking done
Delhi to Kathmandu: River Walks & Epic Wildlife
1225833
Delhi to Kathmandu: River Walks & Epic Wildlife done
Kerala: Beaches & Backwaters
1227310
Kerala: Beaches & Backwaters done
India to Sri Lanka: The Pink City & Spice Gardens
1227319
India to Sri Lanka: The Pink City & Spice Gardens done
Uncover India: High Deserts & Markets
1227320
Uncover India: High Deserts & Markets done
Discover Kerala & Sri Lanka: Islands & Ruins
1227479
Discover Kerala & Sri Lanka: Isla

In [12]:
# df = pd.DataFrame(departures_dictionaries_list)
# df.to_csv('raw_data/one_day_test_v2.csv', index=False)
# df

In [13]:
df = pd.DataFrame(departures_dictionaries_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 28 columns):
 #   Column                                                                     Non-Null Count  Dtype 
---  ------                                                                     --------------  ----- 
 0   tour_name                                                                  25 non-null     object
 1   itinerary_name                                                             25 non-null     object
 2   visited_countries                                                          25 non-null     object
 3   start_date                                                                 25 non-null     object
 4   duration                                                                   25 non-null     int64 
 5   url                                                                        25 non-null     object
 6   itinerary                                                           

In [26]:
pd.set_option('display.max_columns', None)
df.head(50)

,tour_name,itinerary_name,visited_countries,start_date,duration,url,itinerary,Travel Style,Service Level,Physical Grading,Merchandising,Trip Type,Deluxe - Double - D - Adult,Deluxe - Double - D - Adult - Promotion cost,Deluxe - Double - D - Adult - Promotion description,Standard - Twin - D - Adult,Standard - Twin - D - Adult - Promotion cost,Standard - Twin - D - Adult - Promotion description,Standard - Adult,Shared bathroom - Female-only dorm - F - Adult,Shared bathroom - Double/Twin - D - Adult,Shared bathroom - Mixed gender dorm - B - Adult,Shared bathroom - Female-only dorm - F - Adult - Promotion cost,Shared bathroom - Female-only dorm - F - Adult - Promotion description,Shared bathroom - Double/Twin - D - Adult - Promotion cost,Shared bathroom - Double/Twin - D - Adult - Promotion description,Shared bathroom - Mixed gender dorm - B - Adult - Promotion cost,Shared bathroom - Mixed gender dorm - B - Adult - Promotion description
0,Sailing Thailand - Ko Phi Phi to Phuket,,Thailand,2024-02-13,4,https://www.gadventures.com/trips/sailing-thai...,Day 1: Meet in Ko Phi Phi in the early afterno...,Marine - Small-ship adventures on the world’s ...,Standard - Comfortable tourist-class accommoda...,3 - Average - Some tours may include light hik...,Book Your Bubble - The Book Your Bubble Collec...,Small Group - Group trips average 12 traveller...,899.00,719.20,20% off Sailing,799.00,639.20,20% off Sailing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Masai Mara Camping Safari,,Kenya,2024-02-13,4,https://www.gadventures.com/trips/masai-mara-c...,Day 1: Arrive at any time.\nDay 2: Journey thr...,"Classic - All of the highlights, culture, acce...",Camping - Camping most nights with some hotel/...,2 - Light - Light walking and hiking suitable ...,Planeterra Project,Small Group - Group trips average 12 traveller...,NaN,NaN,NaN,NaN,NaN,NaN,1079.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Indochina Discovery: Seasides and Street Food,,"Vietnam, Laos, Cambodia, Thailand",2024-02-13,30,https://www.gadventures.com/trips/indochina-di...,Day 1: Arrive at any time and welcome to Thail...,"18-to-Thirtysomethings - Fast, fresh, and fun ...",Basic - Simple and clean hotels and hostels; a...,2 - Light - Light walking and hiking suitable ...,"Top Seller, Planeterra Project",Small Group - Group trips average 12 traveller...,NaN,NaN,NaN,NaN,NaN,NaN,2299.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Torres del Paine - The W Trek,,Chile,2024-02-13,6,https://www.gadventures.com/trips/torres-del-p...,Day 1: Arrive at any time.\nDay 2: Embark on a...,"Active - Hiking, trekking, biking, rafting, an...",Camping - Camping most nights with some hotel/...,4 - Demanding - Some high-altitude hikes or mo...,Book Your Bubble - The Book Your Bubble Collec...,Small Group - Group trips average 12 traveller...,NaN,NaN,NaN,NaN,NaN,NaN,2399.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hike Patagonia In Depth,,"Argentina, Chile",2024-02-13,14,https://www.gadventures.com/trips/hike-patagon...,Day 1: Arrive at any time.\nDay 2: Fly to El C...,"Active - Hiking, trekking, biking, rafting, an...",Standard - Comfortable tourist-class accommoda...,4 - Demanding - Some high-altitude hikes or mo...,Book Your Bubble - The Book Your Bubble Collec...,Small Group - Group trips average 12 traveller...,NaN,NaN,NaN,NaN,NaN,NaN,4859.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Patagonia Hiking,,Argentina,2024-02-13,9,https://www.gadventures.com/trips/patagonia-hi...,Day 1: Arrive at any time.\nDay 2: Fly to El C...,"Active - Hiking, trekking, biking, rafting, an...",Standard - Comfortable tourist-class accommoda...,4 - Demanding - Some high-altitude hikes or mo...,NaN,Small Group - Group trips average 12 traveller...,NaN,NaN,NaN,NaN,NaN,NaN,2619.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Delhi to Kathmandu: River Walks & Epic Wildlife,,"India, Nepal",2024-02-13,15,https://www.gadventures.com/trips/delhi-to-kat...,Day 1: Arrive at any time and welcome to India...,"18-to-Thirtysomethings - Fast, fresh, and fun ...

In [15]:
list(countries_set)

['Egypt',
 'Sri Lanka',
 'Argentina',
 'Chile',
 'Jordan',
 'Kenya',
 'Laos',
 'Morocco',
 'Costa Rica',
 'Peru',
 'Cambodia',
 'Thailand',
 'India',
 'Australia',
 'Vietnam',
 'Nepal']

In [23]:
value = filtered_departures.first()

In [24]:
if value.availability['status'] == "AVAILABLE" and value.tour_dossier != None: 
    trip_list = [category['name'] for category in value.tour_dossier.categories if category['category_type']['label'] ==  "Trip Type"]
    if 'Small Group' in trip_list:
        for i in range(len(value.structured_itineraries)):

            # get tour name
            tour_name = value.tour_dossier.name
            print(tour_name)
            print(value.id)

            # structured itinerary id
            structured_itineraries_name = value.structured_itineraries[i].name

            # get the list of countries visited
            visited_countries_list = []
            for country in value.tour_dossier.geography['visited_countries']:
                visited_countries_list.append(country["name"])
                countries_set.add(country["name"]) #  add countries to set


            # get start date
            start_date = value.start_date

            # get duration
            duration = value.finish_date - value.start_date + timedelta(days=1)
            duration = duration.days

            # get pricing form room categories
            room_categories_dictionaries = {}

            for room in value.rooms:
                data = room.to_dict()
                for price_brand in data["price_bands"]:
                    for price in price_brand["prices"]:
                        if price['currency'] == "GBP":
                            # get prices
                            price_column = f'{data["name"]} - {price_brand["name"]}'
                            if price_column in room_categories_dictionaries:
                                room_categories_dictionaries[price_column] += price['amount']
                            else:
                                room_categories_dictionaries[price_column] = price['amount']
                            # get promotion
                            if  len(price['promotions']) > 0:
                                print("started promo")
                                # get promo cost
                                promotion_column_cost = f'{data["name"]} - {price_brand["name"]} - Promotion cost'
                                if promotion_column_cost in room_categories_dictionaries:
                                    room_categories_dictionaries[promotion_column_cost] += price["promotions"][0]['amount']
                                else:
                                    room_categories_dictionaries[promotion_column_cost] = price["promotions"][0]['amount']
                                # promo cost done
                                # get promo description
                                promotion_column_name = f'{data["name"]} - {price_brand["name"]} - Promotion description'
                                if promotion_column_name in room_categories_dictionaries:
                                    room_categories_dictionaries[promotion_column_name] += client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                else:
                                    room_categories_dictionaries[promotion_column_name] = client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                print("finished promo")

            # get tour url 
            tour_url = value.tour_dossier.site_links[0]['href']

            # get itinerary day by day
            itinerary = ""
            itinerary_days = value.structured_itineraries[i]
            for day in itinerary_days.days:
                itinerary += f'Day {day.day}: {day.summary}\n'

            # get tour categories
            categories_id_list = [i['id'] for i in value.tour_dossier.categories]

            tour_categories_dictionaries = {}

            for category in categories_id_list:
                category_dict = client.tour_categories.get(category).to_dict()
                category_value = None
                if category_dict['description'] == None:
                    category_value = category_dict['name']
                else:
                    category_value = f"{category_dict['name']} - {category_dict['description']}"

                if category_dict['category_type']['name'] in tour_categories_dictionaries:
                    tour_categories_dictionaries[category_dict['category_type']['name']] += ", " + category_value
                else:
                    tour_categories_dictionaries[category_dict['category_type']['name']] = category_value

            # create dictionary
            dataframe_dictionary = {}
            dataframe_dictionary['tour_name'] = tour_name
            dataframe_dictionary['itinerary_name'] = structured_itineraries_name
            dataframe_dictionary['visited_countries'] = ", ".join(visited_countries_list)
            dataframe_dictionary['start_date'] = start_date
            dataframe_dictionary['duration'] = duration
            dataframe_dictionary['url'] = tour_url
            dataframe_dictionary['itinerary'] = itinerary


            # combine dictionaries
            dataframe_dictionary = dict(dataframe_dictionary, **tour_categories_dictionaries, **room_categories_dictionaries)



            # add full dict to list
            departures_dictionaries_list.append(dataframe_dictionary)
            print(tour_name + " " + "done")

Sailing Thailand - Ko Phi Phi to Phuket
1131977
started promo
finished promo
started promo
finished promo
Sailing Thailand - Ko Phi Phi to Phuket done


In [25]:
value.rooms

[<DepartureRoom (Deluxe - Double - D)>, <DepartureRoom (Standard - Twin - D)>]